## 1. Importar las Librerías Necesarias

In [ ]:
## Integrantes: Federico Méndez Casanave y Pablo Marenco.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from wordcloud import WordCloud
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from xgboost import XGBClassifier
from sklearn.metrics import cohen_kappa_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

%matplotlib inline
pd.set_option('display.max_columns', 30)
plt.rcParams['figure.figsize'] = [12.0, 8.0]

## 2. Leer los Datos
Al menos los datos Tabulares de la base de "train"

In [ ]:
#Importamos el dataset
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')

In [ ]:
train.head()

## 3. Pre-procesar Nulos
Verificar la existencia de Nulos y decidir como Imputarlos en caso de que existan

Verificar la existencia de Ceros u otros valores que puedan indicar que pueden ser perdidos

In [ ]:
#Identificamos variables con valores nulos
train.isnull().sum()

# Únicamente 'Name' y 'Description' tienen nulos, por lo que no las incorporaremos al modelo.

In [ ]:
train["Name"].value_counts()

In [ ]:
train['Description'].value_counts()

In [ ]:
# Observamos valores únicos por cada columna
train.nunique()

In [ ]:
# Creamos dummies que indican si las variables 'Name' y 'Description' tienen valores.
train['Name_Yes'] = np.where(train['Name'].notnull(), 0, 1)
train['Description_Yes'] = np.where(train['Description'].notnull(), 0, 1)

In [ ]:
train.info()

In [ ]:
# Estadísticas descriptivas
train.describe()

In [ ]:
# Matriz de correlaciones de variables numéricas
df_adoptado =train.copy()
df_adoptado['AdoptionSpeed'].replace(
    to_replace=[0, 1, 2 , 3],
    value=0,
    inplace=True
)
df_adoptado['AdoptionSpeed'].replace(
    to_replace=[4],
    value=1,
    inplace=True
)

f, ax1 = plt.subplots(1,1,figsize=(20,12))

corr=train.corr()

mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(240, 10, as_cmap=True,n=9)

ax1=sns.heatmap(corr, mask=mask, cmap=cmap,  center=0,
            square=True, linewidths=.5,vmax=0.3, cbar_kws={"shrink": .9})
ax1.set_title("AdoptionSpeed",fontsize=14)

In [ ]:
# Adopciones según especie de animal
ax = sns.countplot(x="Type", data=train)
sns.set(font_scale=1.0)
ax.set(xlabel = 'Animal', ylabel = 'Total')
ax.set_xticklabels(["Perros","Gatos"])
for i,p in enumerate(ax.patches):
    ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()-80),
                ha='center', va='center', fontsize=10, color='black', xytext=(0, 10),
                textcoords='offset points')

In [ ]:
train.head()

In [ ]:
train.info()

## 4. Convertir o eliminar las Columnas Categóricas

Por ejemplo, la Descripción habría que sacarla para un análisis independiente

In [ ]:
# Eliminamos las variables categóricas innecesarias
To_drop = ["Name","RescuerID","Description","PetID"]
data = train.drop(To_drop, axis=1)
print(train.columns)

## 5. Normalizar o Estandarizar las variables Numericas (para los modelos que sean necesarios)

Revisar si existen valores extremos y considerarlos para los modelos que afecte

In [ ]:
# No es necesaria ninguna normalización de variables numéricas para aplicar LightGBM.

## 6. Separa la base de Test (10%) y Train (90%)
Pueden ser otros porcentajes que les parezcan mejor

In [ ]:
X= data.loc[:, data.columns != 'AdoptionSpeed']
y= data['AdoptionSpeed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2021)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
y_train.head()

In [ ]:
y_test

### 7. Para la parte de Train, armar un esquema de Cross Validation

Usar 10 Folds

In [ ]:
# Definitimos la métrica de kappa
def metric(y_true, y_pred):
    res = cohen_kappa_score(y_true, y_pred.reshape((y_true.shape[0], 5), order="F").argmax(axis=1), weights= 'quadratic')
    return "kappa", res, True

In [ ]:
pipeline = Pipeline([("classifier",LGBMClassifier(n_estimators=10, metric="custom",verbose=50, early_stopping_rounds=20) )])

In [ ]:
busqueda = [
    {
        "classifier": [LGBMClassifier(n_estimators=100, metric="custom")],
        "classifier__max_depth": list(range(5, 12, 1)),
        "classifier__min_data_in_leaf": list(range(38,50,1)),
        "classifier__learning_rate": [0.1,0.15,0.2]
    }
]
classifier = GridSearchCV(pipeline, busqueda, cv=10, verbose=30)

## 8. Entrenar al menos un Modelo que prefieran y optimizar al menos un Hiperparámetro

In [ ]:
best_model = classifier.fit(X_train, y_train)
print(best_model.best_estimator_.get_params()["classifier"])

In [ ]:
#Importamos el dataset de test
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')

In [ ]:
test.head()

In [ ]:
# Aplicamos el mismo procesamiento que se había aplicado al dataset de train
test['Name_Yes'] = np.where(test['Name'].notnull(), 0, 1)
test['Description_Yes'] = np.where(test['Description'].notnull(), 0, 1)

In [ ]:
To_drop2 = ["Name","RescuerID","Description","PetID"]
test2 = test.drop(To_drop, axis=1)
print(test.columns)

In [ ]:
# Fiteamos predicciones del mejor modelo al dataset de test
predicciones = best_model.predict(test2)

In [ ]:
# Creamos submission
submission = test[['PetID']].copy()
submission["AdoptionSpeed"] = predicciones.astype("int64")
submission.to_csv("submission.csv", index=False)

submission["AdoptionSpeed"].value_counts(normalize=True,dropna=False)